# Decompression and Prediction for pre-defined U/A
#### (Using the Tonsil dataset)

This script gives an example on how to read in a compressed CISI experiment, how to decompress it and how to evaluate results using a pre-defined U and A/Phi using the data from the proof-of-concept study with an antibody-mix compressing eigth to six/four channels.

## Setup

### Import Libraries

First we set up the script by specifying the necessary libraries, importing the functions from CISI for IMC and specifying input paths and parameters used by CISI for IMC.

In [2]:
# Import libraries
import anndata as ad
from pathlib import Path
import errno
import os
import numpy as np
import sys
import pandas as pd
from scipy.spatial import distance


# Helper fncs
import helpers.analysis_utils

In [3]:
## CISI
# Configure code directory as module

# Find code directory relative to our directory
THIS_DIR = os.path.dirname('__file__')
CODE_DIR = os.path.abspath(os.path.join(THIS_DIR, '..', 'code'))
# Add code directory to systems paths
sys.path.append(CODE_DIR)

# Import CISI training fnc.
from analyze_dictionary_and_compositions import analyze_U_and_A
from decompress import decompress
from utils import compare_results, get_observations_no_noise

### Specify Paths

In [4]:
## Specify input paths
data_path = Path('/mnt/bb_dqbm_volume')
training_path = Path(os.path.join(data_path,
                                'data/Tonsil_th152/preprocessed_data/spe.h5ad'))

'''
experiment_path = Path(os.path.join(data_path,
                                'data/20221108_TsH_LSS_cisiabmix2_179/preprocessed_data/composite_measurements.h5ad'))

U_path = Path(os.path.join(data_path,
                                'analysis/Tonsil_th179_2/gene_modules.csv'))
A_path = Path(os.path.join(data_path,
                                'analysis/Tonsil_th179_2/version_1.txt'))
A_best_path = Path(os.path.join(data_path,
                                'analysis/Tonsil_th179_2/version_best.txt'))

'''
experiment_path = Path(os.path.join(data_path,
                                'data/20221108_TsH_LSS_cisiabmix1_179/preprocessed_data/composite_measurements.h5ad'))

U_path = Path(os.path.join(data_path,
                           'analysis/Tonsil_th179_1/gene_modules.csv'))
A_path = Path(os.path.join(data_path,
                           'analysis/Tonsil_th179_1/version_1.txt'))
A_best_path = Path(os.path.join(data_path,
                                'analysis/Tonsil_th179_1/version_best.txt'))


## Specify output path
# out_path = Path(os.path.join(data_path, 'analysis/Tonsil_th179_2'))
out_path = Path(os.path.join(data_path, 'analysis/Tonsil_th179_1'))
out_path_training = Path(os.path.join(out_path, 'training'))
out_path_experiment = Path(os.path.join(out_path, 'experiment'))
out_path_experiment_simulation = Path(os.path.join(out_path, 'experiment-simulation'))
out_path_experiment_simulation_best_A = Path(os.path.join(out_path, 'experiment-simulation-best-A'))
# Create output directory if it doesn't exist
out_path_training.mkdir(parents=True, exist_ok=True)
out_path_experiment.mkdir(parents=True, exist_ok=True)
out_path_experiment_simulation.mkdir(parents=True, exist_ok=True)
out_path_experiment_simulation_best_A.mkdir(parents=True, exist_ok=True)

In [6]:
# Check that input files/dictionary exist
if not helpers.analysis_utils.is_valid_file(training_path, ['.h5ad']):
    # If file is not found, throw error
    raise FileNotFoundError(errno.ENOENT, os.strerror(errno.ENOENT),
                            training_path)
if not helpers.analysis_utils.is_valid_file(experiment_path, ['.h5ad']):
    # If file is not found, throw error
    raise FileNotFoundError(errno.ENOENT, os.strerror(errno.ENOENT),
                            experiment_path)    
if not helpers.analysis_utils.is_valid_file(U_path, ['.csv']):
    # If file is not found, throw error
    raise FileNotFoundError(errno.ENOENT, os.strerror(errno.ENOENT),
                            U_path)
if not helpers.analysis_utils.is_valid_file(A_path, ['.txt']):
    # If file is not found, throw error
    raise FileNotFoundError(errno.ENOENT, os.strerror(errno.ENOENT),
                            A_path)

if not helpers.analysis_utils.is_valid_file(A_best_path, ['.txt']):
    # If file is not found, throw error
    raise FileNotFoundError(errno.ENOENT, os.strerror(errno.ENOENT),
                            A_best_path)

### Specify parameters

In [7]:
## Specify parameters for CISI
dictionary_size = 17   # 10
normalization = 'none'

# Define test rois
test_names_training = ('20220520_TsH_th152_cisi1_002',)

## Read Inputs

Next, we read in all inputs (anndata objects containing composite measurements and training data, U, A/Phi, best A predicted by parameter sweep) and make sure that the composite channels and proteins are all in the same order for U, A/Phi and anndata objects to ensure correct matrix multiplications in CISI for IMC.

In [8]:
## Read in data
# Read in SingleCellExperiment converted to anndata by cellconverter in R
sce_training = ad.read_h5ad(training_path)
sce_experiment = ad.read_h5ad(experiment_path)
# Channels of interest
channels_of_interest = sce_experiment.var_names[sce_experiment.var_names.str.contains('^CC\d_', regex=True)]
proteins_of_interest = sce_experiment.var_names[~sce_experiment.var_names.isin(channels_of_interest)]
proteins_of_interest = proteins_of_interest[~proteins_of_interest.str.contains('^Ir', regex=True)]
# Remove uninteresting proteins/channels
sce_training = sce_training[:, proteins_of_interest.tolist()]

# Read U
U = np.genfromtxt(U_path, delimiter=',', skip_header=True,
                         usecols=list(range(1, (dictionary_size)+1)))
U_names = [x.decode() for x in np.genfromtxt(U_path, delimiter=',', usecols=0, skip_header=1,
                                             dtype='S20')]

# Read A
A = np.loadtxt(A_path, skiprows=1, usecols=list(range(1, len(proteins_of_interest)+1)))
A_names = [x.decode() for x in np.loadtxt(A_path, max_rows=1, dtype='S20')]
A_channels = [x.decode() for x in np.loadtxt(A_path, usecols=0, skiprows=1, dtype='S20')]


# Read best A
A_best = np.loadtxt(A_best_path, skiprows=1, usecols=list(range(1, len(proteins_of_interest)+1)))
A_best_names = [x.decode() for x in np.loadtxt(A_best_path, max_rows=1, dtype='S20')]
A_best_channels = [x.decode() for x in np.loadtxt(A_best_path, usecols=0, skiprows=1, dtype='S20')]

In [7]:
## Throw error if A and U don't have the same proteins or A has different channels than the specified channels
## of interest
if (all(e in U_names for e in A_names) & all(e in A_names for e in U_names) &
   all(e in A_channels for e in channels_of_interest) & all(e in channels_of_interest for e in A_channels)):
    U_index = [U_names.index(ind) for ind in proteins_of_interest]
    A_index = [A_names.index(ind) for ind in proteins_of_interest]
    A_channel_index = [A_channels.index(ind) for ind in channels_of_interest]
    A_best_index = [A_best_names.index(ind) for ind in proteins_of_interest]
    
    U = U[U_index, :].copy()
    A = A[:, A_index].copy()
    A = A[A_channel_index, :].copy()
    A_best = A_best[:, A_best_index].copy()
    
else:
    # Throw error
    raise ValueError(('A and U do not have the same proteins or A does ' +
                      'not have the channels of interest.' +
                      'Please check all files, to prevent CISI ' +
                      'from computing wrong results.\n' +
                      'Proteins in U: {0}'.format(U_names) +
                      'Proteins in A: {0}\n'.format(A_names) +
                      'Channels in A: {0}\n'.format(A_channels)))

## Predict Performance

For the training data and the individually measured proteins in the composite experiment, we simulate composite measurements and evaluate their perfomance using the pre-defined and read in U and A/Phi. For the real experiment we also simulate composite measurements and evaluate the performance using the best A from the parameter run.

In [8]:
## Predict performance of A given A,U and SCE
# Test for training data and real experiment data

(predicted_res_training, 
 predicted_res_noisy_training) = analyze_U_and_A(sce_training[sce_training.obs['sample_id'].isin(test_names_training), ],
                                                 U, [A], ['none'], out_path_training,
                                                 norm=normalization)
(predicted_res_experiment, 
 predicted_res_noisy_experiment) = analyze_U_and_A(sce_experiment[:, sce_experiment.var.index.isin(proteins_of_interest)],
                                                   U, [A], ['none'], out_path_experiment_simulation,
                                                   norm=normalization)

(predicted_res_experiment_best, 
 predicted_res_noisy_experiment_best) = analyze_U_and_A(sce_experiment[:, sce_experiment.var.index.isin(proteins_of_interest)],
                                                        U, [A_best], ['none'], out_path_experiment_simulation_best_A,
                                                        norm=normalization)

/home/ubuntu/anaconda3/envs/cisi_imc_env/lib/python3.10/site-packages/scipy/spatial/distance.py:630: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)
/home/ubuntu/anaconda3/envs/cisi_imc_env/lib/python3.10/site-packages/scipy/spatial/distance.py:630: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)
/home/ubuntu/anaconda3/envs/cisi_imc_env/lib/python3.10/site-packages/scipy/spatial/distance.py:630: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)


In [ ]:
# Write simulated values to anndata
y_experiment = get_observations_no_noise((sce_experiment[:, sce_experiment.var.index.isin(proteins_of_interest)].X).T,
                                          A, normalization=normalization)

## TODO: change name
# Write simulated composite measurements from the experiment to anndata
composite_anndata = sce_experiment[:, sce_experiment.var.index.isin(channels_of_interest)].copy()
for k in list(composite_anndata.layers.keys()):
    del composite_anndata.layers[k]
    composite_anndata.X = y_experiment.T
    composite_anndata.write(os.path.join(out_path_experiment_simulation, 'simulated_composite_measurements.h5ad'))

## Decompression

In the last part we do the actual decomposition of composite measurements and evaluate their performance compared to the individually measured protein levels as is done in the above function analyze_U_and_A().

In [ ]:
## Decompression
# Decompress composite channels                     
decompressed_x = decompress((sce_experiment[:, sce_experiment.var.index.isin(channels_of_interest)].X).T, 
                            U, A)
# Remove infinit values                   
decompressed_x[np.isnan(decompressed_x)] = 0
# Compute statistics
decompression_results = compare_results((sce_experiment[:, sce_experiment.var.index.isin(proteins_of_interest)].X).T, 
                                        decompressed_x)
d_gene = np.percentile(1 - distance.pdist(A.dot(U).T, 'correlation'), 90)

# Open file to save decompression results                 
decompression_file = open(os.path.join(out_path_experiment, 'simulation_results.txt'), 'w')
colnames = ['version', 'Overall pearson', 'Overall spearman', 'Gene average',
            'Sample average', 'Sample dist pearson', 'Sample dist spearman',
            'Gene dist pearson', 'Gene dist spearman',
            'Matrix coherence (90th ptile)']
decompression_file.write('\t'.join(colnames) + '\n')                     
decompression_file.write('\t'.join([str(x) for x in ['']+decompression_results+[d_gene]]) + '\n')
decompression_file.close()

In [ ]:
# Write decomposed X to anndata
decompressed_anndata = sce_experiment[:, sce_experiment.var.index.isin(proteins_of_interest)].copy()
for k in list(decompressed_anndata.layers.keys()):
    del decompressed_anndata.layers[k]
    decompressed_anndata.X = decompressed_x.T
    decompressed_anndata.write(os.path.join(out_path_experiment, 'X_decomposed.h5ad'))

# Write original X subseted to individual protein expression levels to the same place as the decomposed X
(sce_experiment[:, sce_experiment.var.index.isin(proteins_of_interest)]).write(os.path.join(out_path_experiment, 'X_test.h5ad'))